<h1>
<center>
Assignment 1: Regression
</center>
</h1>
<center>
CS 4262/5262 - Foundations of Machine Learning<br>
Vanderbilt University, Spring 2023<br>
Due: Check Brightspace
</center>
<hr>
<br>In this first assignment, you will have a chance to implement linear and polynomial regression models. In addition to programming tasks (marked with #TODO), there are some short-answer questions throughout the notebook. Please do not hesitate to ask for clarification. 

### Please enter your name:  Manda Li


---
## Part 1: Dataset
We will be using a data set describing California housing. The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000). 

The data set contains several input features, including the median income, the average number of rooms, as well as average number of bedrooms, in the census block group. There are a large number of outliers in this group since many of the units were unoccupied due to their being hotels or other units at the time of the census. Thus, we will use the median income as the main feature we track (Note: the units for median income are not needed for this assignment, but are likely not in $100,000s!).

This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

An household is a group of people residing within a home. Since the average number of rooms and bedrooms in this dataset are provided per household, these columns may take surpinsingly large values for block groups with few households and many empty houses, such as vacation resorts.

You can learn more about the data here: https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import random
import sklearn.datasets 
import time

In [ ]:
'''
Fetches data of interest: median California house price for an average number of rooms (in the 1990s).
@return median household income (numpy.ndarray), median price of house (numpy.ndarray)
'''
def fetch_housing_data():
    from sklearn.datasets import fetch_california_housing
    housing_data = fetch_california_housing()
    median_income = housing_data.data[:,0]
    median_price = housing_data.target
    return median_income, median_price 


'''
Renders a simple (x,y) plot. 
@param x (numpy.ndarray) - median income of group (census values)
       y (numpy.ndarray) - median price of house 
'''
def simple_plot(x,y):
    plt.scatter(x,y)
    plt.title('California Housing')
    plt.xlabel('Median Income of Group')
    plt.ylabel('Median Price ($100,000s)')
    plt.show()

**Run the two functions above to display a plot of the data. Based on the plot, do you think linear regression will be able to effectively model the data? Why or why not?**

Response:<br>Yes,because the plot shows a possible linear correlation between median income and median price (when x increases, y increases). The dataset will be likely to be well described by a linear regression model.


In [ ]:
#TODO
# Run the functions here to set values for the variables median_income, and median_price
median_income, median_price = fetch_housing_data()
simple_plot(median_income,median_price)


The `split_train_test()` function below will split the data into a training set and test set. Run the cell (no need to call the function yet).

**What is the definition of overfitting? Why is it necessary to have both training and test sets?**

Response:<br> Overfitting is the trained model does well only on the training set and not generally applicable in predicting the results out of the training set, having test sets will give us a view of how well our model does outside the training set and avoid possible overfitting. 


In [ ]:
'''
Partitions a dataset into a training set and a test set (80/20 split, respectively). 
Shuffles dataset before splitting. 
@param x - data inputs (numpy.ndarray)
       y - data targets (numpy.ndarray)
@return training set inputs, 
        training set targets, 
        test set inputs,
        test set targets (all numpy.ndarray)
'''
def split_train_test(x, y):
    xy = list(zip(x, y))
    random.shuffle(xy)
    x, y = zip(*xy)
    
    split = int(len(x)*0.8)
    train_x = np.array(x[:split])
    train_y = np.array(y[:split])
    test_x = np.array(x[split:])
    test_y = np.array(y[split:])
    
    train_y = np.reshape(train_y, (len(train_y),1))
    test_y = np.reshape(test_y, (len(test_y),1))
    
    return train_x, train_y, test_x, test_y

---
## Part 2: Simple Linear Regression
For the `SimpleLinearRegressionModel` class below, fill in the incomplete methods using their respective descriptions. All methods you should fill out are marked with a #TODO. This model uses a Mean Squared Error cost function and gradient descent to optimize the parameters. 

In [ ]:
class SimpleLinearRegressionModel():
    
    '''
    Implementation of simple linear regression using gradient descent
    @param x (numpy.ndarray) - training set of 'single-feature' inputs
           y (numpy.ndarray) - training set of corresponding targets
           theta (numpy.ndarray) - model parameters in the form of [intercept, coefficient]
           alpha (float) - gradient descent step size
    '''
    def __init__(self, x, y, theta, alpha):
        self.x = x
        self.y = y
        self.theta = theta
        self.alpha = alpha
     
    '''
    Equation for the regression line. 
    input x_i (float) - single input feature
    @return corresponding model output (float)
    '''
    #TODO 
    def h(self, x_i):
        x_0=1
        x_in=[x_0,x_i]
        #print(self.theta, x_in)
        h_func= np.multiply(x_in, self.theta)
        h_result=np.sum(h_func)
        return h_result
    
    '''
    Renders a plot of the training data and the regression line based on current model parameters.
    ''' 
    def plot_current_model(self):
        reg_line_y = [self.h(x_i) for x_i in self.x]
        plt.scatter(self.x, self.y)
        plt.plot(self.x, reg_line_y, color='green')
        plt.title('California Housing')
        plt.xlabel('Average Number of Rooms')
        plt.ylabel('Median Price ($100,000s)')
        plt.show()
    
    '''
    Loss function measuring mean squared error of the regression line for a given training set and model parameters. 
    @return MSE based on the current parameters (float)
    '''
    #TODO
    def J(self):
        m=float(len(self.x))
        h_x = [self.h(x_i) for x_i in self.x]
        error=np.subtract(h_x, self.y)
        J_func=(1/(2*m))*(np.sum(error*error))
        return J_func
    
    '''
    Update the model parameters (i.e. the two theta values) for one gradient descent step. Hint: this involves 
    computing partial derivatives. 
    '''
    #TODO
    def gradient_descent_step(self):
        h_x = [self.h(x_i) for x_i in self.x]
        #print("h_x size:", np.shape(h_x))
        error=np.subtract(h_x, self.y)
        #print("y_size:",np.shape(self.y))
        x_0=1
        #xi=x_0+self.x
        m=float(len(self.x))
        #print("Length:",m)
        theta0=self.theta[0]
        #print("1st theta 0:",theta0)
        theta1=self.theta[1]
        rest= error*self.x
        #print("Rest size", np.size(rest))
        #print("Sum Error*x",np.sum(error*self.x))
        #print("1/m:",(1/m)*np.sum(error*self.x) )
        theta1= theta1- self.alpha*((1/m)*np.sum(np.multiply(error,self.x)))
        theta0= theta0- self.alpha*((1/m)*np.sum(error))
        result=[theta0,theta1]
        #print("theta result after step:",result)
        return result
        
        
    '''
    Run gradient descent to optimize the model parameters.
    Keep track of the value of the cost function. You may change the default threshold for convergence.
    @param threshold (float) - run gradient descent until the magnitude of the gradient is below this value. 
    @return a list storing the value of the cost function after every step of gradient descent (float list)
    '''
    #TODO
    def run_gradient_descent(self, threshold=0.001):
        #theta=self.theta
        self.plot_current_model()
        theta_history=[]
        
        loss=[]
        previous_loss=None
        for i in range(100000):
            h_x=[self.h(x_i) for x_i in self.x]
            current_loss= self.J()
            print("Loss:", current_loss)
            if previous_loss and abs(current_loss - previous_loss)<=threshold:
                break
            previous_loss=current_loss
            loss.append(current_loss)
            theta_history.append(self.theta)
            
            self.theta=self.gradient_descent_step()
            print("Updated theta:", self.theta)
            print("Iteration:",i)
        self.plot_current_model()
            
            
        return loss
    
    '''
    Renders plot of MSE at each iteration of gradient descent
    @param losses (float list) - MSE after every gradient descent step 
    '''
    def plot_MSE_loss(self, losses):
        plt.plot(range(len(losses)), losses)
        plt.title('Learning Curve')
        plt.xlabel('Number of Steps')
        plt.ylabel('MSE Loss')
        plt.show()

Before applying this model to the California housing dataset, let's test your code on a smaller dataset. In the debugging function below, first create a small set of 25 training examples ((x,y) pairs) that have an approximately linear relationship. You can do so by first generating pairs that have a perfect linear relationship, then adding a bit of random noise (as much magnitude as you like). Then, run your Simple Linear Regression model on this dataset and display the plot of the final regression line, superimposed on the 25 datapoints (call the `plot_current_model()` method). 

In [ ]:
'''
Function for debugging the simple linear regression model.
'''
#TODO
def debug_SLR_model():
    #sample=fetch_housing_data()
    sample_income=median_income[:25]
    sample_price=median_price[:25]
    #print(sample_income[0])
    #simple_plot(sample_income,sample_price)
    alpha_test=0.01
    theta_test=[0,0.5]
    sample_model=SimpleLinearRegressionModel(sample_income,sample_price,theta_test,alpha_test)
    loss=sample_model.run_gradient_descent()
    
    sample_model.plot_MSE_loss(loss)
    return

debug_SLR_model()

If your plot seems to run through your data, then your model looks like it'll work. Next, partition the California housing data into a training set and a test set (use the `split_train_test()` function from Part 1). 

Once you have partitioned the data, run the training set through new instances of `SimpleLinearRegression`. Try a value within the [$-5$,$5$] range for $\theta_0$ and a value within the [$-0.25$,$1$] range for $\theta_1$.

Use the `plot_MSE_loss()` method to display the learning curve for three different step sizes (alpha): 0.01, 0.1, and 0.5.

**How do the learning curves differ? What can you infer about the effect of changing alpha, particularly as it increases/decreases? Are there values of alpha that do not seem to work? Does your initial guess for $\theta$ seem to influence values of alpha? Can you modify either alpha or theta to get better results?**

Response:<br> The learning curve converged faster with learning rate at 0.01. When learning rate is 0.01, the theta values bounced less in the case and reach the gal faster in around 16 iterations with theta[0,1], faster than the other 2 learning rates.The selection of theta will influence the value of alpha because larger alpha helps converge faster when theta is not too close to the goal like[5,1] 


In [ ]:
# Assess the different step sizes here (next four cells).
train_x, train_y, test_x, test_y = split_train_test(median_income,median_price)
train_y=np.reshape(train_y,(16512,))
print("Train y",train_y)
#train_y=np.reshape(train_y,(16512,0))

In [ ]:
alpha1 = 0.01
#TODO - run an instance of the model and plot the learning curve
theta_1=[5,-0.25]

first_model=SimpleLinearRegressionModel(train_x,train_y,theta_1,alpha1)
loss=first_model.run_gradient_descent()
first_model.plot_MSE_loss(loss)



In [ ]:
alpha2 = 0.1
theta_1=[5,1]
#TODO
first_model=SimpleLinearRegressionModel(train_x,train_y,theta_1,alpha2)
loss=first_model.run_gradient_descent()
first_model.plot_MSE_loss(loss)


In [ ]:
alpha3 = 0.5
theta1=[0,1]
#TODO
first_model=SimpleLinearRegressionModel(train_x,train_y,theta_1,alpha2)
loss=first_model.run_gradient_descent()
first_model.plot_MSE_loss(loss)


In [ ]:
print(first_model.theta)

In [ ]:
# Evaluate model on test data here (next two cells). 

'''
Make predictions for inputs using a trained SLR model. 
@param model (SimpleLinearRegression) - SLR model with (ideally) optimized parameters
       test_x (numpy.ndarray) - inputs to run through the model 
@return corresponding predictions for the inputs (numpy.ndarray)
'''
#TODO
def SLR_predict(model, test_x):
    y_predict= [x_i * model.theta[1] +model.theta[0] for x_i in test_x]
    #print(y_predict)
    return y_predict

#SLR_predict(first_model,test_x)
'''
Renders a plot showing the actual vs SLR-predicted outputs for a set of inputs. 
@param test_x (numpy.ndarray) - model inputs
       test_y (numpy.ndarray) - corresponding actual outputs 
       pred_y (numpy.ndarray) - corresponding predicted outputs
'''
#TODO
def plot_actual_vs_pred(test_x, test_y, pred_y):
    #plt.plot(test_x, test_y, color='green')
    plt.scatter(test_x, test_y)
    plt.plot(test_x, pred_y, color='orange')
    plt.title('California Housing - Predict vs Actual')
    plt.xlabel('Average Number of Rooms')
    plt.ylabel('Median Price ($100,000s)')
    plt.show()
    return

In [ ]:
# TODO
# replace with your best starting guess, from above cells
theta = [0,1]
# replace with the fastest alpha that you got above
alpha = 0.01

slr = SimpleLinearRegressionModel(train_x, train_y, theta, alpha)
start_time = time.time()
losses = slr.run_gradient_descent()
end_time = time.time()
print(end_time - start_time)
pred_y = SLR_predict(slr, test_x)
plot_actual_vs_pred(test_x, test_y, pred_y)



---
## Part 3: Polynomial Regression 

In Part 2, we modeled the data with a linear function. Now, we will build a polynomial regression model and apply it to the same California housing data. We will implement gradient descent, and vectorize operations whenever possible.

In class, we learned that the gradient descent update rule can be written as:
   $\theta := \theta - \alpha \cdot \nabla J(\theta)$,
   and that for Batch Gradient Descent, the $j^{th}$ entry of $\nabla_\theta J(\theta)$ can be expressed as:
 
   $$\sum_{i=1}^m (h_\theta(x^{(i)})-y^{(i)}) \cdot x_j^{(i)} $$

In the `PolynomialRegressionModel` class below, note that there is a new field `self.degree` in the `init()` method, which specifies the highest polynomial degree in your regression equation. For example, if `self.degree=3`, then the regression equation is $y_i = \theta_0 + \theta_1 x_i + \theta_2 x_i^2 + \theta_3 x_i^3$.

**Although we are fitting a nonlinear function to our data, why can polynomial regression be considered a case of multiple linear regression?***

Response:<br> Because we can see the value of (x1)^2 as a new feature like x2.


<br>**First,** complete the function `form_design_matrix()` below. This function should return the design matrix $X$ for polynomial regression, which is a matrix of dimensions $m \times (d+1)$, where $d$ is the degree of the polynomial. Remember to include the intercept term (i.e., append the 'feature' $x_0 = 1$ to each training example, such that the first column of $X$ consists of all 1's. 

In [ ]:
'''Converts an array of training set inputs into a design matrix, where rows represent 
training inputs and columns represent input features. 
@param training_inputs (numpy.ndarray) - training set of input features
       degree (int) - highest polynomial degree to extend the design matrix into
@return design matrix including the x_0 'feature' - (numpy.ndarray)
'''
#TODO
def form_design_matrix(training_inputs, degree):
    X=[]
    for x in training_inputs:
        new_row=[]
        for i in range(0,degree+1):
            new_row.append(np.power(x,i))
            #new_col=new_col.reshape(len(new_col))
        #print(new_row)
        X.append(new_row)
        #X=np.reshape(16512,4)
    #print(X)
    print(np.shape(X))
    X=np.array(X)
    print(type(X))
    print(np.shape(X))
    return X

form_design_matrix(train_x, 3)

**Next,** before writing any further code, show that the above expression for the jth entry of the gradient ( = $\sum_{i=1}^m (h_\theta(x^{(i)})-y^{(i)}) \cdot x_j^{(i)} $) indeed corresponds to the jth entry of $\nabla_\theta J(\theta) = X^T(X\theta - y)$. You can write your derivation below using LaTeX, or upload a separate sheet of paper with a hand-written derivation. 

In [ ]:
#TODO
# Show your gradient derivation here, or upload derivation as a separate document.

**Then,** complete the methods in the `PolynomialRegressionModel` class using vectorization. Assume that the model's training input will be in the form of a design matrix. A lot of your SLR code can be copied over to this new class— modify your methods accordingly to work for any $d$-degree polynomial. 

In [ ]:
class PolynomialRegressionModel():
    
    '''
    Implementation of a Polynomial regression model using MSE and Gradient Descent.
    @param X (numpy.ndarray) - training set of input features, as a design matrix
           y (numpy.ndarray) - training set of corresponding targets
           theta (numpy.ndarray) - model parameters (variable coefficients, in order of increasing variable degree)
           alpha (float) - learning step size
           degree (int) - highest polynomial degree
    '''
    def __init__(self, X, y, theta, alpha, degree):
        self.X = X
        self.y = y
        self.theta = theta
        self.alpha = alpha
        self.degree = degree
             
    '''
    Hypothesis - return model prediction 
    @param X (numpy.ndarray) - design matrix of input features
    @return corresponding model output (float)
    '''
    #TODO
    def h(self, X):
        print(X)
        result=np.dot(X,self.theta)
        return result
    
    '''
    Renders a plot of the training data and the regression line based on current model parameters.
    ''' 
    def plot_current_model(self):
        reg_curve_y = self.h(self.X)
        plt.scatter(self.X[:,1], self.y)
        plt.scatter(self.X[:,1], reg_curve_y, color='green')
        plt.title('California Housing (scaled data)')
        plt.xlabel('Median Income')
        plt.ylabel('Median Price')
        plt.show()
    
    '''
    Cost function measuring mean squared error of the regression line for a given training set and model parameters.
    Vectorize your code - no loops!
    @return MSE based on the current parameters (float)
    '''
    #TODO
    def J(self):
        m,n=np.shape(self.X)
        h_x = self.h(self.X)
        error=h_x - self.y
        J_func=(1/(2*m))*np.sum((error*error))
        return J_func
    
    '''
    Update theta for one gradient descent step. Vectorize your code - no loops!
    @return the gradient of the cost function (numpy.ndarray), for use in run_gradient_descent
    '''
    #TODO
    def gradient_descent_step(self):
        
        m,n=np.shape(self.X)
        h_x = self.h(self.X)
        error=h_x - self.y
        dev=(1/m)*np.dot(self.X.T,error)
        self.theta= self.theta- self.alpha*dev
        return self.theta
       
    '''
    Run gradient descent to optimize the model parameters.
    Keep track of the losses. You may change the default threshold for convergence. 
    Here, we will use a convergence criterion based on the norm of the gradient vector.
    @param threshold (float) - run gradient descent until the absolute norm of the gradient is below this value.
    @return a list storing the value of the cost function after every step of gradient descent (float list)
    '''
    #TODO: add debug statements to show progress if you want (but limit printouts for readability)
    def run_gradient_descent(self, threshold=0.01):
        losses = []
        loss = self.J()
        losses.append(loss)
        self.plot_current_model()
        
        norm_grad_vec = 1
        while norm_grad_vec > threshold:
            grad_vec = self.gradient_descent_step()
            loss = self.J()
            losses.append(loss)
            norm_grad_vec = np.linalg.norm(grad_vec)
            #self.plot_current_model()
        
        return losses
    
    '''
    Renders the learning curve of the model during its optmization process. 
    @param losses (float list) - MSE after every gradient descent step 
    '''
    def plot_MSE_loss(self, losses):
        plt.plot(range(len(losses)), losses)
        plt.title('Learning Curve')
        plt.xlabel('Number of Steps')
        plt.ylabel('MSE Loss')
        plt.show()

Next, we will run an instance of the `PolynomialRegressionModel` class with a degree of 3. As feature scaling will help our model to converge more quickly, we can first scale our design matrix (normalize each column to have zero mean, unit variance - except for the column of ones). Here, let's set up the design matrix and perform this feature scaling (you can just execute this code, you do not need to write anything here - but make sure you understand what the code is doing). The resulting scaled design matrix will be your input to `PolynomialRegressionModel` in the next part.

In [ ]:
degree = 3
#print(train_x)
design_mat = form_design_matrix(train_x, degree)
print(np.shape(design_mat))
Xz = sc.stats.zscore(design_mat[:,1:]) # scale *except* for the column of ones
m = Xz.shape[0]
ones_vec = np.ones((m,1))
design_mat_scaled = np.hstack((ones_vec,Xz))

**Now,** run an instance of the `PolynomialRegressionModel` class using this design matrix. Experiment with the parameter 'alpha' and the initial value of the vector 'theta'. Plot the data and resulting regression curve using the method `plot_current_model`. 

In [ ]:
#TODO
# Run an instance of the Polynomial Regression Model with a degree of 3 on the California housing data here. 

# select a starting theta that will converge quickly, as you do this homework refine your best guess
theta_init = np.array([20, -80, 160, -80.0]) 
theta_init = np.reshape(theta_init, (degree+1,1))
# now call the PRM class you created, and plot the losses and final regression curve
prm=PolynomialRegressionModel(design_mat,train_y,theta_init, alpha, degree)
loss=prm.run_gradient_descent()
prm.plot_MSE_loss(loss)


**What do you foresee will happen if you apply this model to the held-out test set?**

Response:<br> This model could have a more accurate result.


**Lastly,** as an alternative to running gradient descent for linear regression, one can solve directly for the value of $\theta$ that minimizes the least-squares cost function $J(\theta)$, using the equation:
$$ \theta = (X^T X)^{-1} X^T y $$

Please fill in the method `closed_form_solution()` below, using this equation. Run this using the feature-scaled design matrix constructed above, and report the value of $\theta$ it returns, and compare it to the solution you obtained using gradient descent. 

Note:  your predictions will be the same regardless of whether you use the scaled or non-scaled version of the design matrix (optionally, you can check this on your own!). However, for the sake of comparing the values of $\theta$, you can use the feature-scaled design matrix here.

In [ ]:
'''
Equation for directly solving for theta which minimizes the least-squares cost function J(theta).
@param X (numpy.ndarray) - model input in the form of a design matrix 
       y (numpy.ndarray) - model output 
@return set of theta values for the regression equation that minimizes J(theta) (numpy.ndarray)
'''
#TODO 
def closed_form_solution(X, y):
    theta=np.linalg.inv(X.T @ X) @ X.T @ y
    return theta

In [ ]:
#TODO - run closed_form_solution()
X_new=form_design_matrix(train_x, degree)
print(type(X_new))
theta=closed_form_solution(X_new,train_y)
print(theta)

**Compare gradient descent theta to closed-form solution theta. When you guess an initial value "close" to the closed-form solution, do you think your gradient descent value recovers the closed form version?**<br>

Response:<br> It depends on the choice of learning rate,if learning rate is not too large gradient descent value will recover the closed form version fast.


**In what scenario(s) might one choose to use gradient descent over solving the closed-form solution?**

Response:<br>When the data set is too large to comput its design matrix transpose and the time limit is loose enough for gradient descent training.


---
## Part 4: Submission 

Once you're ready to submit, create a 'clean' version of your final solutions, removing any extra debugging code you may have written. Next, in the menu bar, click `Kernel > Restart & Clear Output`. Then run your code from top to bottom, so that all the plots are displayed. Back in the menu bar, click `File > Download as > Notebook (.ipynb)` to download your notebook. Don't forget to answer the short answer questions. 

Please upload this to Brightspace by midnight of the deadline. 